# Scrapování obrázků z webu

Protože kdo by chtěl stahovat hambaté obrázky manuálně.

In [ ]:
import os
from IPython import get_ipython
from IPython.display import display, Markdown

COLAB = True

if COLAB:
  from google.colab.output import clear as clear_output
else:
  from IPython.display import clear_output

#@title ## 🚩 Start Here

#@markdown ### 1️⃣ Setup
#@markdown This cell will load some requirements and create the necessary folders in your Google Drive. <p>
#@markdown Your project name can't contain spaces but it can contain a single / to make a subfolder in your dataset.
project_name = "summoning_circle" #@param {type:"string"}
project_name = project_name.strip()
#@markdown The folder structure doesn't matter and is purely for comfort. Make sure to always pick the same one. I like organizing by project.
folder_structure = "Organize by project (MyDrive/Loras/project_name/dataset)" #@param ["Organize by category (MyDrive/lora_training/datasets/project_name)", "Organize by project (MyDrive/Loras/project_name/dataset)"]

if not project_name or any(c in project_name for c in " .()\"'\\") or project_name.count("/") > 1:
  print("Please write a valid project_name.")
else:
  if COLAB and not os.path.exists('/content/drive'):
    from google.colab import drive
    print("📂 Connecting to Google Drive...")
    drive.mount('/content/drive')

  project_base = project_name if "/" not in project_name else project_name[:project_name.rfind("/")]
  project_subfolder = project_name if "/" not in project_name else project_name[project_name.rfind("/")+1:]

  root_dir = "/content" if COLAB else "~/Loras"
  deps_dir = os.path.join(root_dir, "deps")

  if "/Loras" in folder_structure:
    main_dir      = os.path.join(root_dir, "drive/MyDrive/Loras") if COLAB else root_dir
    config_folder = os.path.join(main_dir, project_base)
    images_folder = os.path.join(main_dir, project_base, "dataset")
    if "/" in project_name:
      images_folder = os.path.join(images_folder, project_subfolder)
  else:
    main_dir      = os.path.join(root_dir, "drive/MyDrive/lora_training") if COLAB else root_dir
    config_folder = os.path.join(main_dir, "config", project_name)
    images_folder = os.path.join(main_dir, "datasets", project_name)

  for dir in [main_dir, deps_dir, images_folder, config_folder]:
    os.makedirs(dir, exist_ok=True)

  print(f"✅ Project {project_name} is ready!")
  step1_installed_flag = True


✅ Project summoning_circle is ready!


#Zde je příklad, jak kombinovat klíčová slova:

#Pro hledání obrázků, které obsahují alespoň jedno z klíčových slov: Použijte operátor OR (mezera). Například search_term = "Arial Rebel Nude OR Boudoir" vyhledá obrázky, které obsahují buď "Arial Rebel Nude", nebo "Boudoir".
#Pro hledání obrázků, které obsahují všechna klíčová slova: Použijte operátor AND (znaménko plus). Například search_term = "Arial Rebel Nude + Boudoir" vyhledá obrázky, které obsahují "Arial Rebel Nude" i "Boudoir".

#Kombinovat klíčová slova můžete libovolně pomocí těchto operátorů. Například search_term = "(Arial Rebel Nude OR Boudoir) + Portrait" vyhledá obrázky, které obsahují "Arial Rebel Nude" nebo "Boudoir" a zároveň "Portrait".

#Můžete také použít závorky, abyste definovali pořadí operací. Například search_term = "(Arial Rebel Nude AND Boudoir) OR Portrait" vyhledá obrázky, které obsahují "Arial Rebel Nude" a "Boudoir" nebo "Portrait".

#teoreticky lze použít i site: na hledání přímo na určitých stránkách.

In [ ]:
#@markdown ### 2️⃣ Sosání obrázků z netu

#@markdown čapne obrázky z webu podle zadaných kritérií


!pip install bing_image_downloader

from bing_image_downloader import downloader
from IPython.display import Image, display
import os
import hashlib

import ipywidgets as widgets
from IPython.display import display

# Widgety pro zadávání klíčových slov
keyword_input = widgets.Text(
    value='',
    placeholder='Zadejte klíčové slovo',
    description='Klíčové slovo:',
    disabled=False
)

add_keyword_button = widgets.Button(
    description='Přidat klíčové slovo',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Přidá klíčové slovo do seznamu',
    icon='plus' # (FontAwesome names without the `fa-` prefix)
)

keyword_list = widgets.SelectMultiple(
    options=[],
    value=[],
    rows=10,
    description='Klíčová slova:',
    disabled=False
)

# Funkce pro přidání klíčového slova
def add_keyword(b):
    keyword = keyword_input.value.strip()
    if keyword and keyword not in keyword_list.options:
        keyword_list.options = keyword_list.options + (keyword,)
        keyword_input.value = ''

# Připojení funkce k tlačítku
add_keyword_button.on_click(add_keyword)

# Zobrazení widgetů
display(keyword_input, add_keyword_button, keyword_list)

# Vytvoření proměnné search_term z vybraných klíčových slov
search_term = ' + '.join(keyword_list.value)

print(f"search_term: {search_term}") # Toto zobrazí aktuální search_term

search_term = "site: www.deviantart.com + summoning circle" #@param {type:"string"}
limit = 1000 #@param {type:"slider", min:1, max:1000, step:1}
#@markdown Use negative keywords to exclude unwanted images. Separate them with spaces.
negative_keywords = "tattoo" #@param {type:"string"}
#@markdown Enter a specific URL to scrape images from that website only. Leave empty to search Bing.
# New parameter for minimum image size in KB
min_image_size_kb = 500  #@param {type:"number"}
specific_url = "" #@param {type:"string"}
#@markdown Set the minimum resolution for downloaded images. Format: widthxheight (e.g., 1920x1080). Leave empty to disable.
resolution = "" #@param {type:"string"}
#@markdown disable filter.
adult_filter = True #@param {type:"boolean"} #This line does not matter, see below.

def download_images_bing():
    print (f"🔽 Will download to {images_folder.replace('/content/drive/', '')} (A confirmation box should appear below, otherwise run this cell again)")
    inp = input("❓ Enter the word 'yes' if you want to proceed with the download: ")

    if inp.lower().strip() != 'yes':
        print("❌ Download cancelled")
        return

    downloader.download(
        search_term,
        limit=limit,
        output_dir=images_folder,
        adult_filter_off=adult_filter,
        force_replace=False,
        timeout=60,
        verbose=True
    )
    print(f"\n✅ Downloaded {limit} images.")

    # Preview downloaded images
    print("\n🖼️ Preview of downloaded images:")
    image_files = [f for f in os.listdir(images_folder) if os.path.isfile(os.path.join(images_folder, f))]
    for i in range(min(10, len(image_files))):
        display(Image(filename=os.path.join(images_folder, image_files[i])))

    # Eliminate duplicates
    print("\n🧹 Eliminating duplicates...")
    unique_files = []
    duplicates = []
    for filename in image_files:
        filepath = os.path.join(images_folder, filename)
        with open(filepath, 'rb') as f:
            filehash = hashlib.md5(f.read()).hexdigest()
        if filehash not in unique_files:
            unique_files.append(filehash)
        else:
            duplicates.append(filepath)
            os.remove(filepath)

    print(f"🗑️ Removed {len(duplicates)} duplicate images.")

download_images_bing()

Text(value='', description='Klíčové slovo:', placeholder='Zadejte klíčové slovo')

Button(description='Přidat klíčové slovo', icon='plus', style=ButtonStyle(), tooltip='Přidá klíčové slovo do s…

SelectMultiple(description='Klíčová slova:', options=(), rows=10, value=())

Streaming output truncated to the last 5000 lines.


[!!]Indexing page: 2284

[%] Indexed 4 Images on Page 2284.




[!!]Indexing page: 2285

[%] Indexed 4 Images on Page 2285.




[!!]Indexing page: 2286

[%] Indexed 4 Images on Page 2286.




[!!]Indexing page: 2287

[%] Indexed 4 Images on Page 2287.




[!!]Indexing page: 2288

[%] Indexed 4 Images on Page 2288.




[!!]Indexing page: 2289

[%] Indexed 4 Images on Page 2289.




[!!]Indexing page: 2290

[%] Indexed 4 Images on Page 2290.




[!!]Indexing page: 2291

[%] Indexed 4 Images on Page 2291.




[!!]Indexing page: 2292

[%] Indexed 4 Images on Page 2292.




[!!]Indexing page: 2293

[%] Indexed 4 Images on Page 2293.




[!!]Indexing page: 2294

[%] Indexed 4 Images on Page 2294.




[!!]Indexing page: 2295

[%] Indexed 4 Images on Page 2295.




[!!]Indexing page: 2296

[%] Indexed 4 Images on Page 2296.




[!!]Indexing page: 2297

[%] Indexed 4 Images on Page 2297.




[!!]Indexing page: 2298

[%] Indexed 4 Images on Pa

KeyboardInterrupt: 

**Nepovinné: Možnost fiknout obrázky na 1:1**

In [ ]:
from PIL import Image

def crop_and_resize(image_path, output_size):
    """Ořízne a změní velikost obrázku do poměru 1:1."""
    try:
        img = Image.open(image_path)
        width, height = img.size

        # Oříznutí do čtverce
        if width > height:
            left = (width - height) / 2
            top = 0
            right = (width + height) / 2
            bottom = height
        else:
            left = 0
            top = (height - width) / 2
            right = width
            bottom = (height + width) / 2

        img = img.crop((left, top, right, bottom))

        # Změna velikosti
        img = img.resize((output_size, output_size), Image.Resampling.LANCZOS)

        # Uložení obrázku
        img.save(image_path)

        print(f"✅ Obrázek {image_path} oříznut a změněna velikost.")
    except Exception as e:
        print(f"❌ Chyba při zpracování obrázku {image_path}: {e}")

# Použití funkce pro všechny obrázky v adresáři
def process_images(images_folder, output_size):
    """Zpracuje všechny obrázky v daném adresáři."""
    for filename in os.listdir(images_folder):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join(images_folder, filename)
            crop_and_resize(image_path, output_size)

# Nastavení velikosti výstupního obrázku
output_size = 512  # Zadejte požadovanou velikost

# Zpracování obrázků
process_images(images_folder, output_size)

In [ ]:
if "step1_installed_flag" not in globals():
  raise Exception("Please run step 1 first!")

#@markdown ### Nepovinné: Zčoučni co to soslo
#@markdown protože jsem idiot, tak je nutné jít manuálně do složky s obrázky, přesunout je nad dataset, dataset smazat a poté složku s obrázky přejmenovat na dataset. Poté vše funguje a
#@markdown FilthyOne teda FiftyOne najde duplicitní obrázky a eliminuje je. <p>
#@markdown Tohle je citlivost na duplicitu - cca 0.97 až 0.99:
similarity_threshold = 0.985 #@param {type:"number"}


os.chdir(root_dir)
model_name = "clip-vit-base32-torch"
supported_types = (".png", ".jpg", ".jpeg")
img_count = len(os.listdir(images_folder))
batch_size = min(250, img_count)

if "step3_installed_flag" not in globals():
  print("🏭 Installing dependencies...\n")
  !pip -q install fiftyone ftfy
  !pip -q install fiftyone-db-ubuntu2204
  if not get_ipython().__dict__['user_ns']['_exit_code']:
    clear_output()
    step3_installed_flag = True
  else:
    print("❌ Error installing dependencies, attempting to continue anyway...")

import numpy as np
import fiftyone as fo
import fiftyone.zoo as foz
from fiftyone import ViewField as F
from sklearn.metrics.pairwise import cosine_similarity

non_images = [f for f in os.listdir(images_folder) if not f.lower().endswith(supported_types)]
if non_images:
  print(f"💥 Error: Found non-image file {non_images[0]} - This program doesn't allow it. Sorry! Use the Extras at the bottom to clean the folder.")
elif img_count == 0:
  print(f"💥 Error: No images found in {images_folder}")
else:
  print("\n💿 Analyzing dataset...\n")
  dataset = fo.Dataset.from_dir(images_folder, dataset_type=fo.types.ImageDirectory)
  model = foz.load_zoo_model(model_name)
  embeddings = dataset.compute_embeddings(model, batch_size=batch_size)

  batch_embeddings = np.array_split(embeddings, batch_size)
  similarity_matrices = []
  max_size_x = max(array.shape[0] for array in batch_embeddings)
  max_size_y = max(array.shape[1] for array in batch_embeddings)

  for i, batch_embedding in enumerate(batch_embeddings):
    similarity = cosine_similarity(batch_embedding)
    #Pad 0 for np.concatenate
    padded_array = np.zeros((max_size_x, max_size_y))
    padded_array[0:similarity.shape[0], 0:similarity.shape[1]] = similarity
    similarity_matrices.append(padded_array)

  similarity_matrix = np.concatenate(similarity_matrices, axis=0)
  similarity_matrix = similarity_matrix[0:embeddings.shape[0], 0:embeddings.shape[0]]

  similarity_matrix = cosine_similarity(embeddings)
  similarity_matrix -= np.identity(len(similarity_matrix))

  dataset.match(F("max_similarity") > similarity_threshold)
  dataset.tags = ["delete", "has_duplicates"]

  id_map = [s.id for s in dataset.select_fields(["id"])]
  samples_to_remove = set()
  samples_to_keep = set()

  for idx, sample in enumerate(dataset):
    if sample.id not in samples_to_remove:
      # Keep the first instance of two duplicates
      samples_to_keep.add(sample.id)

      dup_idxs = np.where(similarity_matrix[idx] > similarity_threshold)[0]
      for dup in dup_idxs:
          # We kept the first instance so remove all other duplicates
          samples_to_remove.add(id_map[dup])

      if len(dup_idxs) > 0:
          sample.tags.append("has_duplicates")
          sample.save()
    else:
      sample.tags.append("delete")
      sample.save()

  clear_output()

  sidebar_groups = fo.DatasetAppConfig.default_sidebar_groups(dataset)
  for group in sidebar_groups[1:]:
    group.expanded = False
  dataset.app_config.sidebar_groups = sidebar_groups
  dataset.save()
  session = fo.launch_app(dataset)

  print("❗ Wait a minute for the session to load. If it doesn't, read above.")
  print("❗ When it's ready, you'll see a grid of your images.")
  print("❗ On the left side enable \"sample tags\" to visualize the images marked for deletion.")
  print("❗ You can mark your own images with the \"delete\" label by selecting them and pressing the tag icon at the top.")
  input("⭕ When you're done, enter something here to save your changes: ")

  print("💾 Saving...")

  marked = [s for s in dataset if "delete" in s.tags]
  dataset.remove_samples(marked)
  previous_folder = images_folder[:images_folder.rfind("/")]
  dataset.export(export_dir=os.path.join(images_folder, project_subfolder), dataset_type=fo.types.ImageDirectory)

  temp_suffix = "_temp"
  !mv {images_folder} {images_folder}{temp_suffix}
  !mv {images_folder}{temp_suffix}/{project_subfolder} {images_folder}
  !rm -r {images_folder}{temp_suffix}

  session.refresh()
  fo.close_app()
  clear_output()

  print(f"\n✅ Removed {len(marked)} images from dataset. You now have {len(os.listdir(images_folder))} images.")


💥 Error: Found non-image file site: https: - This program doesn't allow it. Sorry! Use the Extras at the bottom to clean the folder.


**zazipovat**

In [ ]:
import os
import zipfile

def zip_images(images_folder, zip_filename):
    """Zazipuje obrázky v daném adresáři do zip souboru."""
    with zipfile.ZipFile(zip_filename, 'w') as zipf:
        for root, _, files in os.walk(images_folder):
            for file in files:
                if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                    zipf.write(os.path.join(root, file), os.path.relpath(os.path.join(root, file), images_folder))

    print(f"✅ Obrázky zazipovány do souboru: {zip_filename}")

# Použití funkce pro zazipování obrázků
images_folder = "/cesta/k/adresari/s/obrazky"  # Zadejte cestu k adresáři s obrázky
zip_filename = "dataset1.zip"  # Zadejte název zip souboru
zip_images(images_folder, zip_filename)

✅ Obrázky zazipovány do souboru: dataset1.zip
